In [ ]:
import numpy as np
import pickle
import healpy as hp
import matplotlib.pyplot as plt
import os
import subprocess
import sys
import scipy
from scipy.optimize import minimize
from scipy import stats
from matplotlib.ticker import FormatStrFormatter
from getdist import plots, MCSamples
import getdist

In [ ]:
font = {'size'   : 20, 'family':'STIXGeneral'}
plt.rcParams.update({
     'text.usetex': True,
     'font.family': 'serif',
     'font.sans-serif': ['Computer Modern'],
     'font.size':20})
plt.rc_context({'axes.autolimit_mode': 'round_numbers'})

getdist.plots.GetDistPlotSettings.legend_fontsize = 14

### To run this notebook, first run all the pipelines with each of the example yaml files, using the same output_dir for all of them. 

In [ ]:
save = True #whether to save plots
Nsims_lfi = 40000 #number of sims used in LFI
tsz_type = 'nongaussian' #either 'gaussian' or 'nongaussian' (whichever was used in the yaml files)
output_dir = '/Users/kristen/Documents/GitHub/NILC-Inference-Pipeline/plotting_notebooks/tests/outputs_90_150' #output_dir that was used in the yaml files



# Load Files

In [ ]:
posteriors_dir = output_dir + '/posteriors'

# LFI
a_mult_lfi = pickle.load(open(f'{posteriors_dir}/a_array_multifrequency_{tsz_type}tsz_{Nsims_lfi//1000}ksims_tszamp150_lfi.p', 'rb'))
a_hilc_wts_once_lfi = pickle.load(open(f'{posteriors_dir}/a_array_HILC_{tsz_type}tsz_weightsonce_{Nsims_lfi//1000}ksims_tszamp150_lfi.p', 'rb'))
a_hilc_wts_vary_lfi = pickle.load(open(f'{posteriors_dir}/a_array_HILC_{tsz_type}tsz_weightsvary_{Nsims_lfi//1000}ksims_tszamp150_lfi.p', 'rb'))
a_nilc_lfi = pickle.load(open(f'{posteriors_dir}/a_array_NILC_{tsz_type}tsz_{Nsims_lfi//1000}ksims_tszamp150_lfi_4scales.p', 'rb'))
recentered_lfi = []
for pipeline in [a_mult_lfi, a_hilc_wts_once_lfi, a_hilc_wts_vary_lfi, a_nilc_lfi]:
    acmb, atsz = pipeline
    recentered_lfi.append(np.array([acmb+(1-np.mean(acmb)), atsz+(1-np.mean(atsz))]))
a_mult_lfi_recentered, a_hilc_wts_once_lfi_recentered, a_hilc_wts_vary_lfi_recentered, a_nilc_lfi_recentered = recentered_lfi

# Gaussian likelihood
a_mult_gaussianlkl = pickle.load(open(f'{posteriors_dir}/a_array_multifrequency_{tsz_type}tsz_2ksims_tszamp150_gaussianlkl.p', 'rb'))
a_hilc_wts_once_analytic = pickle.load(open(f'{posteriors_dir}/a_array_HILC_{tsz_type}tsz_weightsonce_2ksims_tszamp150_gaussianlkl_analytic.p', 'rb'))
a_hilc_wts_once_sr = pickle.load(open(f'{posteriors_dir}/a_array_HILC_{tsz_type}tsz_weightsonce_2ksims_tszamp150_gaussianlkl_sr.p', 'rb'))
a_hilc_wts_vary_gaussianlkl = pickle.load(open(f'{posteriors_dir}/a_array_HILC_{tsz_type}tsz_weightsvary_2ksims_tszamp150_gaussianlkl_sr.p', 'rb'))
a_nilc_gaussianlkl = pickle.load(open(f'{posteriors_dir}/a_array_NILC_{tsz_type}tsz_2ksims_tszamp150_gaussianlkl_4scales.p', 'rb'))


# Create MCSamples Objects

In [ ]:
hilc_wts_once_lfi = a_hilc_wts_once_lfi.T
hilc_wts_vary_lfi = a_hilc_wts_vary_lfi.T
nilc_lfi = a_nilc_lfi.T
mult_lfi = a_mult_lfi.T
hilc_wts_once_lfi_recentered = a_hilc_wts_once_lfi_recentered.T
hilc_wts_vary_lfi_recentered = a_hilc_wts_vary_lfi_recentered.T
nilc_lfi_recentered = a_nilc_lfi_recentered.T
mult_lfi_recentered = a_mult_lfi_recentered.T

names = [r'$A_{\mathrm{CMB}}$', r'$A_{\mathrm{ftSZ}}$']
labels =  names

samples_hilc_wts_once_lfi = MCSamples(samples=hilc_wts_once_lfi, names = names, labels = labels, label='HILC (weights once) LFI')
samples_hilc_wts_vary_lfi = MCSamples(samples=hilc_wts_vary_lfi, names = names, labels = labels, label='HILC (weights vary) LFI')
samples_nilc_lfi = MCSamples(samples=nilc_lfi, names = names, labels = labels, label='NILC LFI')
samples_mult_lfi = MCSamples(samples=mult_lfi, names = names, labels = labels, label='Multifrequency LFI')

samples_hilc_wts_once_lfi_recentered = MCSamples(samples=hilc_wts_once_lfi_recentered, names = names, labels = labels, label='HILC (weights once) LFI (shifted)')
samples_hilc_wts_vary_lfi_recentered = MCSamples(samples=hilc_wts_vary_lfi_recentered, names = names, labels = labels, label='HILC (weights vary) LFI (shifted)')
samples_nilc_lfi_recentered = MCSamples(samples=nilc_lfi_recentered, names = names, labels = labels, label='NILC LFI (shifted)')
samples_mult_lfi_recentered = MCSamples(samples=mult_lfi_recentered, names = names, labels = labels, label='Multifrequency LFI (shifted)')



In [ ]:
hilc_wts_once_analytic = a_hilc_wts_once_analytic.T
hilc_wts_once_sr = a_hilc_wts_once_sr.T
hilc_wts_vary_gaussianlkl = a_hilc_wts_vary_gaussianlkl.T
nilc_gaussianlkl = a_nilc_gaussianlkl.T
mult_gaussianlkl = a_mult_gaussianlkl.T


samples_hilc_wts_once_analytic = MCSamples(samples=hilc_wts_once_analytic, names = names, labels = labels, label='HILC (weights once) \nGaussian Likelihood\n(Analytic)')
samples_hilc_wts_once_sr = MCSamples(samples=hilc_wts_once_sr, names = names, labels = labels, label='HILC (weights once) \nGaussian Likelihood (SR)')
samples_hilc_wts_vary_sr = MCSamples(samples=hilc_wts_vary_gaussianlkl, names = names, labels = labels, label='HILC (weights vary) \nGaussian Likelihood')
samples_nilc_gaussianlkl = MCSamples(samples=nilc_gaussianlkl, names = names, labels = labels, label='NILC \nGaussian Likelihood')
samples_mult_gaussianlkl = MCSamples(samples=mult_gaussianlkl, names = names, labels = labels, label='Multifrequency \nGaussian Likelihood')


# Posteriors from Likelihood-Free Inference

In [ ]:
g = plots.get_subplot_plotter()
g.triangle_plot([samples_mult_lfi, samples_hilc_wts_once_lfi, samples_hilc_wts_vary_lfi, samples_nilc_lfi], filled=True)
if save:
    plt.savefig(f'lfi_posteriors_{tsz_type}tsz.pdf')


# Posteriors from Gaussian Likelihood

In [ ]:
g = plots.get_subplot_plotter()
g.triangle_plot([samples_mult_gaussianlkl, samples_hilc_wts_once_analytic, samples_hilc_wts_once_sr, samples_hilc_wts_vary_sr, samples_nilc_gaussianlkl], filled=True)
if save:
    plt.savefig(f'gaussianlkl_posteriors_{tsz_type}tsz.pdf')

# Compare LFI to Gaussian Likelihood

In [ ]:
g = plots.get_subplot_plotter()
g.triangle_plot([samples_mult_gaussianlkl, samples_mult_lfi, samples_mult_lfi_recentered], filled=True)
if save:
    plt.savefig(f'multifrequency_posteriors_{tsz_type}tsz.pdf')

In [ ]:
g = plots.get_subplot_plotter()
g.triangle_plot([samples_hilc_wts_once_analytic, samples_hilc_wts_once_sr, samples_hilc_wts_once_lfi, samples_hilc_wts_once_lfi_recentered], filled=True)
if save:
    plt.savefig(f'hilc_wts_once_posteriors_{tsz_type}tsz.pdf')

In [ ]:
g = plots.get_subplot_plotter()
g.triangle_plot([samples_hilc_wts_vary_sr, samples_hilc_wts_vary_lfi, samples_hilc_wts_vary_lfi_recentered], filled=True)
if save:
    plt.savefig(f'hilc_wts_vary_posteriors_{tsz_type}tsz.pdf')

In [ ]:
g = plots.get_subplot_plotter()
g.triangle_plot([samples_nilc_gaussianlkl, samples_nilc_lfi, samples_nilc_lfi_recentered], filled=True)
if save:
    plt.savefig(f'nilc_posteriors_{tsz_type}tsz.pdf')

# Print Parameter Values for LFI

In [ ]:
for par in [r"$A_{\mathrm{CMB}}$", r'$A_{\mathrm{ftSZ}}$']:
    print(samples_mult_lfi.getInlineLatex(par,limit=1))

In [ ]:
for par in [r"$A_{\mathrm{CMB}}$", r'$A_{\mathrm{ftSZ}}$']:
    print(samples_hilc_wts_once_lfi.getInlineLatex(par,limit=1))

In [ ]:
for par in [r"$A_{\mathrm{CMB}}$", r'$A_{\mathrm{ftSZ}}$']:
    print(samples_hilc_wts_vary_lfi.getInlineLatex(par,limit=1))

In [ ]:
for par in [r"$A_{\mathrm{CMB}}$", r'$A_{\mathrm{ftSZ}}$']:
    print(samples_nilc_lfi.getInlineLatex(par,limit=1))

# Print Parameter Values for Gaussian Likelihood

In [ ]:
for par in [r"$A_{\mathrm{CMB}}$", r'$A_{\mathrm{ftSZ}}$']:
    print(samples_mult_gaussianlkl.getInlineLatex(par,limit=1))

In [ ]:
for par in [r"$A_{\mathrm{CMB}}$", r'$A_{\mathrm{ftSZ}}$']:
    print(samples_hilc_wts_once_analytic.getInlineLatex(par,limit=1))

In [ ]:
for par in [r"$A_{\mathrm{CMB}}$", r'$A_{\mathrm{ftSZ}}$']:
    print(samples_hilc_wts_once_sr.getInlineLatex(par,limit=1))

In [ ]:
for par in [r"$A_{\mathrm{CMB}}$", r'$A_{\mathrm{ftSZ}}$']:
    print(samples_hilc_wts_vary_sr.getInlineLatex(par,limit=1))

In [ ]:
for par in [r"$A_{\mathrm{CMB}}$", r'$A_{\mathrm{ftSZ}}$']:
    print(samples_nilc_gaussianlkl.getInlineLatex(par,limit=1))